    Author: Ganesh Sreeram
   
This file shows a preliminary wokring of how to create a valu investing strategy using the CRSP and COMPUSTAT data base from WRDS. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from pandas.tseries.offsets import MonthEnd

Reading CRSP and Compustat Data (large file was converted into h5 in a seperate file) 

In [2]:
stocks = pd.read_hdf('crsp_monthly_stocks.h5', 'crsp')
cstat  = pd.read_hdf('compustat_annual.h5', 'cstat')
rf = pd.read_csv('risk-free_return.csv', parse_dates=[0], index_col=[0])

In [3]:
stocks.head(5)

,PERMNO,DATE,SHRCD,EXCHCD,SICCD,PRC,VOL,RET,SHROUT
0,10000.0,1986-01-31,10.0,3.0,3990.0,-4.375000,1771.0,NaN,3680.0
1,10000.0,1986-02-28,10.0,3.0,3990.0,-3.250000,828.0,-0.257143,3680.0
2,10000.0,1986-03-31,10.0,3.0,3990.0,-4.437500,1078.0,0.365385,3680.0
3,10000.0,1986-04-30,10.0,3.0,3990.0,-4.000000,957.0,-0.098592,3793.0
4,10000.0,1986-05-30,10.0,3.0,3990.0,-3.109375,1074.0,-0.222656,3793.0


Cleaning CRSP

In [4]:
stocks['DATE'] = stocks['DATE'] + MonthEnd(0)
stocks['PRC']  = np.abs(stocks['PRC'])
stocks['MV'] = stocks['SHROUT']*stocks['PRC']
stocks.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','VOL'], axis=1, inplace=True)
stocks.set_index(['PERMNO','DATE'], inplace=True)
stocks.sort_index(inplace=True)
stocks.head()

RET            MV
PERMNO  DATE                              
10000.0 1986-01-31       NaN  16100.000000
        1986-02-28 -0.257143  11960.000000
        1986-03-31  0.365385  16330.000000
        1986-04-30 -0.098592  15172.000000
        1986-05-31 -0.222656  11793.859375

Now we are looking at Compustat, the data cleaning and organinzing will be similar.

In [5]:
cstat.head(5)

,GVKEY,DATADATE,FYEAR,LPERMNO,CONM,ACT,AT,CEQ,CHE,DLC,...,XINT,XSGA,CAPX,CSHO,CONSOL,INDFMT,DATAFMT,POPSRC,CURCD,COSTAT
0,b'001000',1970-12-31,1970.0,25881.0,b'A & E PLASTIK PAK INC',21.351,33.450,10.544,1.660,12.378,...,0.850,9.420,2.767,2.446,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
1,b'001000',1971-12-31,1971.0,25881.0,b'A & E PLASTIK PAK INC',19.688,29.330,8.381,2.557,2.857,...,1.117,10.548,1.771,2.995,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
2,b'001000',1972-12-31,1972.0,25881.0,b'A & E PLASTIK PAK INC',11.326,19.907,7.021,2.027,0.000,...,0.784,7.551,1.254,2.902,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
3,b'001000',1973-12-31,1973.0,25881.0,b'A & E PLASTIK PAK INC',12.969,21.771,8.567,1.357,0.000,...,0.705,8.532,1.633,2.840,b'C',b'INDL',b'STD',b'D',b'USD',b'I'
4,b'001000',1974-12-31,1974.0,25881.0,b'A & E PLASTIK PAK INC',19.473,25.638,9.843,1.338,0.500,...,0.817,8.859,1.313,2.150,b'C',b'INDL',b'STD',b'D',b'USD',b'I'


In [6]:
#Renaming LPERMNO to PERMNO
cstat.rename(index=str, columns={"LPERMNO":"PERMNO"}, inplace=True)

#Assuming that the data is available after 6 months and all the dates are month end values 
cstat['DATE'] = cstat['DATADATE'] + pd.DateOffset(months=6)
cstat['DATE'] = cstat['DATE'] + MonthEnd(0)
cstat.head(5)

,GVKEY,DATADATE,FYEAR,PERMNO,CONM,ACT,AT,CEQ,CHE,DLC,...,XSGA,CAPX,CSHO,CONSOL,INDFMT,DATAFMT,POPSRC,CURCD,COSTAT,DATE
0,b'001000',1970-12-31,1970.0,25881.0,b'A & E PLASTIK PAK INC',21.351,33.450,10.544,1.660,12.378,...,9.420,2.767,2.446,b'C',b'INDL',b'STD',b'D',b'USD',b'I',1971-06-30
1,b'001000',1971-12-31,1971.0,25881.0,b'A & E PLASTIK PAK INC',19.688,29.330,8.381,2.557,2.857,...,10.548,1.771,2.995,b'C',b'INDL',b'STD',b'D',b'USD',b'I',1972-06-30
2,b'001000',1972-12-31,1972.0,25881.0,b'A & E PLASTIK PAK INC',11.326,19.907,7.021,2.027,0.000,...,7.551,1.254,2.902,b'C',b'INDL',b'STD',b'D',b'USD',b'I',1973-06-30
3,b'001000',1973-12-31,1973.0,25881.0,b'A & E PLASTIK PAK INC',12.969,21.771,8.567,1.357,0.000,...,8.532,1.633,2.840,b'C',b'INDL',b'STD',b'D',b'USD',b'I',1974-06-30
4,b'001000',1974-12-31,1974.0,25881.0,b'A & E PLASTIK PAK INC',19.473,25.638,9.843,1.338,0.500,...,8.859,1.313,2.150,b'C',b'INDL',b'STD',b'D',b'USD',b'I',1975-06-30


In [7]:
#Setting date and PERMNO as the index 
cstat.set_index(['PERMNO','DATE'], inplace=True)
cstat.sort_index(inplace=True)

In [8]:
cstat.head(5)

GVKEY   DATADATE   FYEAR  \
PERMNO  DATE                                       
10000.0 1987-04-30  b'013007' 1986-10-31  1986.0   
10001.0 1986-12-31  b'012994' 1986-06-30  1986.0   
        1987-12-31  b'012994' 1987-06-30  1987.0   
        1988-12-31  b'012994' 1988-06-30  1988.0   
        1989-12-31  b'012994' 1989-06-30  1989.0   

                                               CONM    ACT      AT    CEQ  \
PERMNO  DATE                                                                
10000.0 1987-04-30  b'OPTIMUM MANUFACTURING  -CL A'  1.630   2.115  0.418   
10001.0 1986-12-31               b'GAS NATURAL INC'  2.359  12.242  5.432   
        1987-12-31               b'GAS NATURAL INC'  1.870  11.771  5.369   
        1988-12-31               b'GAS NATURAL INC'  1.987  11.735  5.512   
        1989-12-31               b'GAS NATURAL INC'  3.577  18.565  6.321   

                      CHE    DLC   DLTT   ...     XINT  XSGA   CAPX   CSHO  \
PERMNO  DATE                              ...                                
10000.0 1987-04-30  0.348  0.968  0.058   ...    0.068   1.1  0.240  3.843   
10001.0 1986-12-31  0.746  0.343  2.946   ...    0.332   NaN  0.551  0.985   
        1987-12-31  0.729  0.377  2.750   ...    0.303   NaN  0.513  0.991   
        1988-12-31  0.744  0.325  2.555   ...    0.311   NaN  0.240  0.992   
        1989-12-31  1.177  0.185  7.370   ...    0.651   NaN  0.444  1.001   

                    CONSOL   INDFMT  DATAFMT  POPSRC   CURCD  COSTAT  
PERMNO  DATE                                                          
10000.0 1987-04-30    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
10001.0 1986-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1987-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1988-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1989-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  

[5 rows x 34 columns]

In [9]:
#Eliminating duplicate PERMNO 
stocks = stocks.sort_values(by = ['PERMNO','DATE','MV'], ascending = [True, True, False])
cstat  = cstat.sort_values(by = ['PERMNO','DATE','SEQ'], ascending = [True, True, False])

#Keeping the first observed value 

stocks = stocks[~stocks.index.duplicated(keep='first')]
cstat  = cstat[~cstat.index.duplicated(keep='first')]

#Excess returns 
stocks['XRET'] = stocks['RET'] - rf['RF']

In [10]:
stocks.head(5)

RET            MV      XRET
PERMNO  DATE                                        
10000.0 1986-01-31       NaN  16100.000000       NaN
        1986-02-28 -0.257143  11960.000000 -0.262443
        1986-03-31  0.365385  16330.000000  0.359385
        1986-04-30 -0.098592  15172.000000 -0.103792
        1986-05-31 -0.222656  11793.859375 -0.227556

In [11]:
cstat.head(5)

GVKEY   DATADATE   FYEAR  \
PERMNO  DATE                                       
10000.0 1987-04-30  b'013007' 1986-10-31  1986.0   
10001.0 1986-12-31  b'012994' 1986-06-30  1986.0   
        1987-12-31  b'012994' 1987-06-30  1987.0   
        1988-12-31  b'012994' 1988-06-30  1988.0   
        1989-12-31  b'012994' 1989-06-30  1989.0   

                                               CONM    ACT      AT    CEQ  \
PERMNO  DATE                                                                
10000.0 1987-04-30  b'OPTIMUM MANUFACTURING  -CL A'  1.630   2.115  0.418   
10001.0 1986-12-31               b'GAS NATURAL INC'  2.359  12.242  5.432   
        1987-12-31               b'GAS NATURAL INC'  1.870  11.771  5.369   
        1988-12-31               b'GAS NATURAL INC'  1.987  11.735  5.512   
        1989-12-31               b'GAS NATURAL INC'  3.577  18.565  6.321   

                      CHE    DLC   DLTT   ...     XINT  XSGA   CAPX   CSHO  \
PERMNO  DATE                              ...                                
10000.0 1987-04-30  0.348  0.968  0.058   ...    0.068   1.1  0.240  3.843   
10001.0 1986-12-31  0.746  0.343  2.946   ...    0.332   NaN  0.551  0.985   
        1987-12-31  0.729  0.377  2.750   ...    0.303   NaN  0.513  0.991   
        1988-12-31  0.744  0.325  2.555   ...    0.311   NaN  0.240  0.992   
        1989-12-31  1.177  0.185  7.370   ...    0.651   NaN  0.444  1.001   

                    CONSOL   INDFMT  DATAFMT  POPSRC   CURCD  COSTAT  
PERMNO  DATE                                                          
10000.0 1987-04-30    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
10001.0 1986-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1987-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1988-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  
        1989-12-31    b'C'  b'INDL'   b'STD'    b'D'  b'USD'    b'I'  

[5 rows x 34 columns]

From here on all the assumptions are my own unless mentioned. I will be transferring data from the CSTAT data frame into the Stocks data frame to create our own Value strategy. 

Transferring variables from CSTAT to stocks 

In [12]:
list(cstat.columns.values)

['GVKEY',
 'DATADATE',
 'FYEAR',
 'CONM',
 'ACT',
 'AT',
 'CEQ',
 'CHE',
 'DLC',
 'DLTT',
 'INVT',
 'LCT',
 'LT',
 'PPENT',
 'PSTK',
 'SEQ',
 'TXP',
 'WCAP',
 'COGS',
 'DP',
 'IB',
 'NI',
 'OIBDP',
 'REVT',
 'XINT',
 'XSGA',
 'CAPX',
 'CSHO',
 'CONSOL',
 'INDFMT',
 'DATAFMT',
 'POPSRC',
 'CURCD',
 'COSTAT']

In [13]:
#Current debt  
stocks['current debt'] = cstat['DLC']

#Long Term debt
stocks['long term debt'] = cstat['DLTT']

#Total Stockholders Equity 
stocks['SEQ'] = cstat['SEQ']

#Working Capital 
stocks['IB'] = cstat['IB']

#Depreciation 
stocks['depreciation'] = cstat['DP']

#Net Income
stocks['net income'] = cstat['NI']

#CAPX 
stocks['capx'] = cstat['CAPX']

#Shares O/s
stocks['shares outstanding'] = cstat['CSHO']




Using the fillna pad method to fill the nan values 

In [14]:
#Current debt  
stocks['current debt'] = stocks['current debt'].groupby('PERMNO').fillna(method='pad', limit=15)

#Long Term debt
stocks['long term debt'] = stocks['long term debt'].groupby('PERMNO').fillna(method='pad', limit=15)


#Total Stockholders Equity 
stocks['SEQ'] = stocks['SEQ'].groupby('PERMNO').fillna(method='pad', limit=15)

#IB
stocks['IB'] = stocks['IB'].groupby('PERMNO').fillna(method='pad', limit=15)

#Depreciation 
stocks['depreciation'] = stocks['depreciation'].groupby('PERMNO').fillna(method='pad', limit=15)

#Net Income
stocks['net income'] = stocks['net income'].groupby('PERMNO').fillna(method='pad', limit=15)

#CAPX 
stocks['capx'] = stocks['capx'].groupby('PERMNO').fillna(method='pad', limit=15)

#Shares O/s
stocks['shares outstanding'] = stocks['shares outstanding'].groupby('PERMNO').fillna(method='pad', limit=15)

stocks.head(50)

RET            MV      XRET  current debt  \
PERMNO  DATE                                                         
10000.0 1986-01-31       NaN  16100.000000       NaN           NaN   
        1986-02-28 -0.257143  11960.000000 -0.262443           NaN   
        1986-03-31  0.365385  16330.000000  0.359385           NaN   
        1986-04-30 -0.098592  15172.000000 -0.103792           NaN   
        1986-05-31 -0.222656  11793.859375 -0.227556           NaN   
        1986-06-30 -0.005025  11734.593750 -0.010225           NaN   
        1986-07-31 -0.080808  10786.343750 -0.086008           NaN   
        1986-08-31 -0.615385   4148.593750 -0.619985           NaN   
        1986-09-30 -0.057143   3911.531250 -0.061643           NaN   
        1986-10-31 -0.242424   3002.343750 -0.247024           NaN   
        1986-11-30  0.060000   3182.484375  0.056100           NaN   
        1986-12-31 -0.377358   1981.546875 -0.382258           NaN   
        1987-01-31 -0.212121   1581.531250 -0.216321           NaN   
        1987-02-28  0.000000   1581.531250 -0.004300           NaN   
        1987-03-31 -0.384615    973.250000 -0.389315           NaN   
        1987-04-30 -0.062500    912.421875 -0.066900         0.968   
        1987-05-31 -0.066667    851.593750 -0.070467         0.968   
        1987-06-30       NaN           NaN       NaN         0.968   
10001.0 1986-01-31       NaN   6033.125000       NaN           NaN   
        1986-02-28  0.020408   6156.250000  0.015108           NaN   
        1986-03-31  0.025200   6217.812500  0.019200           NaN   
        1986-04-30  0.009901   6279.375000  0.004701           NaN   
        1986-05-31 -0.009804   6217.812500 -0.014704           NaN   
        1986-06-30 -0.013069   6033.125000 -0.018269           NaN   
        1986-07-31 -0.010204   5971.562500 -0.015404           NaN   
        1986-08-31  0.072165   6402.500000  0.067565           NaN   
        1986-09-30 -0.003077   6317.625000 -0.007577           NaN   
        1986-10-31  0.039216   6565.375000  0.034616           NaN   
        1986-11-30  0.056604   6937.000000  0.052704           NaN   
        1986-12-31  0.015000   6937.000000  0.010100         0.343   
        1987-01-31 -0.035714   6689.250000 -0.039914         0.343   
        1987-02-28 -0.074074   6193.750000 -0.078374         0.343   
        1987-03-31  0.036800   6317.625000  0.032100         0.343   
        1987-04-30 -0.039216   6069.875000 -0.043616         0.343   
        1987-05-31 -0.071429   5636.312500 -0.075229         0.343   
        1987-06-30  0.051429   5822.125000  0.046629         0.343   
        1987-07-31  0.021277   5946.000000  0.016677         0.343   
        1987-08-31  0.083333   6441.500000  0.078633         0.343   
        1987-09-30 -0.022308   6200.000000 -0.026808         0.343   
        1987-10-31  0.020000   6324.000000  0.014000         0.343   
        1987-11-30 -0.029412   6138.000000 -0.032912         0.343   
        1987-12-31 -0.033535   5828.000000 -0.037435         0.377   
        1988-01-31  0.063830   6200.000000  0.060930         0.377   
        1988-02-29  0.080000   6696.000000  0.075400         0.377   
        1988-03-31 -0.076296   6076.000000 -0.080696         0.377   
        1988-04-30  0.030612   6262.000000  0.026012         0.377   
        1988-05-31  0.019802   6386.000000  0.014702         0.377   
        1988-06-30 -0.012039   6200.000000 -0.016939         0.377   
        1988-07-31  0.030000   6386.000000  0.024900         0.377   
        1988-08-31  0.029126   6572.000000  0.023226         0.377   

                    long term debt    SEQ     IB  depreciation  net income  \
PERMNO  DATE                                                                 
10000.0 1986-01-31             NaN    NaN    NaN           NaN         NaN   
        1986-02-28             NaN    NaN    NaN           NaN         NaN   
        1986-03-31             NaN    NaN    NaN           NaN         N

All the remaining Nan values occur because there was no data to pad it with. We will drop all the Nan values. 


In [15]:
stocks.dropna(inplace=True)

Before we can start narrowing down the dataset we will sort it by date and then permno. This will allow us to combine different firms into a single portfolio on the same date.  

In [16]:
stocks = stocks.reorder_levels(['DATE','PERMNO'])
stocks.sort_index(inplace=True)

1. We will narrow down our universe by looking at companies that have a higher CAPX than depriciation. This will indicate if the companies assets are growing at a faster rate than they are depreciating. 

We will also shift the column down 1

We only want companies that maintain a value greater than 0, showing growth

In [17]:

stocks['capxdep'] = (stocks['capx'] - stocks['depreciation']).shift()

#Narrowing down the dataframe 

stocks = stocks.loc[(stocks['capxdep']>0)]

stocks.head(5)
stocks['capxdep'].describe()

count    1.362443e+06
mean     6.358291e+01
std      3.923188e+02
min      1.000000e-03
25%      7.000000e-01
50%      3.561000e+00
75%      1.970000e+01
max      2.379900e+04
Name: capxdep, dtype: float64

As it can be seen in the table above the Min-Max range for "capxdep" are all positive numbers. 

2. Next we will look at the debt/equity ratio. We would like to consider companies that maintain a D/E ratio of 0.5-2.0 

We are doing this to limit our risk exposure and also to minimize the impact of borrowing on earnings. 

In [18]:
stocks['D/E'] = ((stocks['current debt']+stocks['long term debt'])/stocks['SEQ']).shift()

stocks = stocks.loc[(stocks['D/E']<2) & (stocks['D/E']>0.5)]

stocks['D/E'].describe()

count    496170.000000
mean          0.994764
std           0.381869
min           0.500000
25%           0.676224
50%           0.907793
75%           1.239719
max           1.999870
Name: D/E, dtype: float64



3. Finally, we want to invest in companies that have a High ROE and EPS. Since we have already removed companies with extremely low debt and high debt, we should be able to minimize the impact of debt on ROE and EPS. 


In [19]:
stocks['ROE'] = (stocks['net income']/stocks['SEQ']).shift()
stocks['EPS'] = (stocks['net income']/stocks['shares outstanding']).shift()
stocks[['ROE', 'EPS']].describe()

,ROE,EPS
count,496169.000000,496169.000000
mean,-0.022789,1.768101
std,5.887152,52.321522
min,-493.666667,-157.942088
25%,0.007193,0.008163
50%,0.098968,0.761499
75%,0.158635,1.896414
max,1500.500000,7138.690862


Looks like the ROE and EPS measures might have some abnormalities.

In order to avoid those values we will set a range that we like. 

1. ROE : 0.05-1
2. EPS : 1:20

In [20]:
stocks = stocks.loc[(stocks['ROE']>0.02) & (stocks['ROE']<1)]
stocks = stocks.loc[(stocks['EPS']>1) & (stocks['EPS']<20)]

stocks[['ROE', 'EPS']].describe()

,ROE,EPS
count,213503.000000,213503.000000
mean,0.164353,2.643323
std,0.094018,1.825857
min,0.020036,1.000209
25%,0.108383,1.463462
50%,0.142772,2.102957
75%,0.192417,3.162149
max,0.987109,19.958370


Now that we have narrowed down our universe, we will calculate the P/E ratio based on which we will create our portfolio quintiles.

In [26]:
stocks['P/E'] = (stocks['MV'] / stocks['IB']/1000).shift()
stocks = stocks.loc[(stocks['P/E']>5) & (stocks['P/E']<100)]
stocks['P/E'].describe()

count    149691.000000
mean         18.241797
std          15.111592
min           5.000076
25%           8.596733
50%          13.299735
75%          21.597542
max          99.984082
Name: P/E, dtype: float64

Finally, we can break the stocks up into quitiles and implement a buy/sell strategy. 

We will start by dividing the stocks into quitiles using this function: 

In [28]:
def quintiles(inser):
    outser = pd.qcut(inser, q=5, labels=range(1,6))
    return outser

In [29]:
stocks['QUINTILE'] = stocks['P/E'].groupby('DATE').apply(quintiles)

In [33]:
stocks.groupby('QUINTILE')[['P/E', 'ROE', 'EPS','D/E', 'capxdep']].mean()

,P/E,ROE,EPS,D/E,capxdep
QUINTILE,,,,,
1,7.493804,0.156482,2.956518,0.979960,101.028423
2,10.582413,0.151540,2.755199,0.978979,110.115723
3,13.737974,0.157475,2.583633,0.962909,100.271027
4,19.160961,0.166968,2.445448,0.957385,92.898874
5,40.191925,0.170004,2.376858,0.960605,84.630076


As expected the P/E ratio is the highest in the bottom quinitile and so is the ROE. However we can see that the majority of the growing asset companies can be found in quinitile 1-3, which is a good sign. Also, the average D/E is close to 1 throughout, which is also good from a risk perspective. Finally, the earning per share is the highest in quintile 1, which explains the inverse realitionship with P/E. 

In [45]:
portfolio = stocks.groupby(['QUINTILE','DATE'])['XRET'].mean() 
portfolio.head(5)

QUINTILE  DATE      
1         1970-07-31    0.060731
          1970-08-31    0.065274
          1970-09-30    0.175294
          1970-10-31   -0.079656
          1970-11-30   -0.018402
Name: XRET, dtype: float64

Calculating Maximum Drawdown 

In [62]:
#Converting Portfolio into a dataframe 

portfolio1 = pd.DataFrame(stocks.groupby(['QUINTILE','DATE'])['XRET'].mean())

#transferring regular returns 

portfolio1['RET'] = stocks.groupby(['QUINTILE','DATE'])['RET'].mean()

portfolio1.head()

XRET       RET
QUINTILE DATE                          
1        1970-07-31  0.060731  0.065931
         1970-08-31  0.065274  0.070574
         1970-09-30  0.175294  0.180694
         1970-10-31 -0.079656 -0.075056
         1970-11-30 -0.018402 -0.013802

In [75]:
#Maximum Drawdown
portfolio1['minimum'] = np.minimum(portfolio1['XRET'],0)**2
portfolio1['value'] = portfolio1['RET'] + 1 
portfolio1['value'] = portfolio1.groupby('QUINTILE')['value'].cumprod()
portfolio1['cummax'] = portfolio1.groupby('QUINTILE')['value'].cummax()
portfolio1['drawdown']=-(portfolio1['value']/portfolio1['cummax']-1)

In [79]:
#Statistics
stats = portfolio.groupby(['QUINTILE']).describe()
stats['sharpe'] = stats['mean'] / stats['std'] * np.sqrt(12)
stats['max drawdown'] = portfolio1.groupby('QUINTILE')['drawdown'].max()
stats

,count,mean,std,min,25%,50%,75%,max,sharpe,max drawdown
QUINTILE,,,,,,,,,,
1,576.0,0.008969,0.060164,-0.290084,-0.026678,0.008371,0.044719,0.313997,0.516428,0.603734
2,576.0,0.008164,0.059309,-0.307285,-0.022993,0.005638,0.043947,0.285922,0.476834,0.642068
3,576.0,0.010612,0.058906,-0.292492,-0.020680,0.011988,0.043856,0.276593,0.624093,0.622478
4,576.0,0.008771,0.056912,-0.295113,-0.025712,0.008483,0.044183,0.265418,0.533884,0.509473
5,576.0,0.009486,0.060362,-0.320561,-0.024496,0.006611,0.043210,0.292305,0.544387,0.509919


Our Ideal strategy would involve long quintile 3 and short quintile 4. Our intuition behind this is simple, we feel that the companies in quitile 1 do not provide that extra profit making opportunity, considering the risk. 

In [81]:
longshort = portfolio.loc[3] - portfolio.loc[5]
longshort.describe()

count    576.000000
mean       0.001127
std        0.033988
min       -0.159600
25%       -0.016513
50%        0.002061
75%        0.019414
max        0.210129
Name: XRET, dtype: float64